In [1]:
import random

import numpy as np
import pymongo

In [11]:
min_services = 20
max_services = 100

min_acceptable_latency = 1
max_acceptable_latency = 20

min_avg_instructions_per_second = 100000
max_avg_instructions_per_second = 1000000

# Define the number of configurations (n)
n = 100

In [2]:
client_url = "mongodb://localhost:27017/"
db_name = "simulations"
collection_name = "services"

In [13]:
# Initialize a dictionary to store the generated service configurations
service_configurations = []

# Connect to MongoDB
client = pymongo.MongoClient(client_url)
db = client[db_name]
collection = db[collection_name] 

layer_list = ['edge', 'fog', 'cloud'] + [None] * 20

# Generate service configurations for each network configuration
for i in range(n):
    config_id = f'config-{i + 1}'  # Unique configuration ID for the network configuration
    
    # Generate a random number of services (m) within the defined range
    num_services = np.random.randint(min_services, max_services + 1)
    
    service_configs = []
    for j in range(num_services):
        service_id = f'service-{j + 1}'  # Unique service ID
        
        # Generate attributes for the service
        acceptable_latency = np.random.uniform(min_acceptable_latency, max_acceptable_latency)
        avg_instructions_per_second = np.random.uniform(min_avg_instructions_per_second, max_avg_instructions_per_second)
        layer = random.choice(layer_list)
        
        # Create the service configuration as a dictionary
        service_config = {
            '_id': service_id,
            'acceptable_latency': acceptable_latency,
            'average_instructions_per_second': avg_instructions_per_second,
            'layer': layer
        }
        
        service_configs.append(service_config)
    
    # Create the service configuration for the network configuration
    network_service_config = {
        '_id': config_id,
        'services': service_configs
    }
    
    # Insert the service configuration into MongoDB
    collection.insert_one(network_service_config)
    service_configurations.append(network_service_config)

# Close the MongoDB connection
client.close()

In [3]:
from services import ServiceConfigurationManager

service_config_manager = ServiceConfigurationManager(client_url, db_name, collection_name)

In [4]:
all_service_configs = service_config_manager.get_all_configurations()
for i, config in enumerate(all_service_configs, start=1):
    print(f"Service Configuration {i}:")
    for service in config.services:
        print(f"Service {service.id}:")
        print(f"Acceptable Latency: {service.acceptable_latency}")
        print(f"Average Instructions per Second: {service.average_instructions_per_second}")
        print(f"Layer: {service.layer}")

Service Configuration 1:
Service service-1:
Acceptable Latency: 17.852142830531974
Average Instructions per Second: 668465.5100419314
Layer: fog
Service service-2:
Acceptable Latency: 1.4069866104751068
Average Instructions per Second: 214909.3261587695
Layer: None
Service service-3:
Acceptable Latency: 14.867504784074852
Average Instructions per Second: 430269.5900536933
Layer: None
Service service-4:
Acceptable Latency: 3.558407802773834
Average Instructions per Second: 547809.8085637549
Layer: None
Service service-5:
Acceptable Latency: 19.68613250564375
Average Instructions per Second: 157335.76838307967
Layer: None
Service service-6:
Acceptable Latency: 3.744038802977441
Average Instructions per Second: 128925.8575801941
Layer: None
Service service-7:
Acceptable Latency: 16.665485768550088
Average Instructions per Second: 931184.6355121095
Layer: None
Service service-8:
Acceptable Latency: 11.155060974050048
Average Instructions per Second: 284505.6514754195
Layer: None
Service se